In [28]:
import face_recognition
from PIL import Image
import os
import glob
import sys
import csv
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
import numpy
from sklearn.metrics import accuracy_score


In [2]:
# Setup face lists
face_1 = face_recognition.load_image_file("./faces/1/subject01 5.png")
face_2 = face_recognition.load_image_file("./faces/2/subject02 11.PNG")
face_3 = face_recognition.load_image_file("./faces/3/subject03 4.png")
face_4 = face_recognition.load_image_file("./faces/4/subject04 4.png")
face_5 = face_recognition.load_image_file("./faces/5/subject05 4.png")
face_6 = face_recognition.load_image_file("./faces/6/subject06 4.png")
face_7 = face_recognition.load_image_file("./faces/7/subject07 4.png")
face_8 = face_recognition.load_image_file("./faces/8/subject08 4.png")
face_9 = face_recognition.load_image_file("./faces/9/subject09 4.png")
face_10 = face_recognition.load_image_file("./faces/10/subject10 4.png")
face_11 = face_recognition.load_image_file("./faces/11/subject11 4.png")
face_12 = face_recognition.load_image_file("./faces/12/subject12 4.png")
face_13 = face_recognition.load_image_file("./faces/13/subject13 4.png")
face_14 = face_recognition.load_image_file("./faces/14/subject14 4.png")
face_15 = face_recognition.load_image_file("./faces/15/subject15 4.png")

label_faces = [face_1,face_2,face_3,face_4,face_5,face_6, face_7, face_8, face_9, face_10, face_11,face_12,face_13,face_14,face_15]


list_of_files = []
for (dirpath, dirnames, filenames) in os.walk("./faces/all"):
    list_of_files.extend(filenames)

path = "/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces/all/"
list_of_files = [path + x for x in list_of_files]
try:
    list_of_files.remove("/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces/all/.DS_Store")
except:
    None


number_files = len(list_of_files)
names = list_of_files.copy()

unknown_faces = []
for x in list_of_files:
    unknown_faces.append(face_recognition.load_image_file(x))
    

In [3]:
# Facial Recognition

labelled_face_encodings = []
unknown_encodings = []

for x in label_faces:
    try:
        labelled_face_encodings.append(face_recognition.face_encodings(x)[0])
    except IndexError:
        print("No face detected.")
        continue

for x in unknown_faces:
    try:
        unknown_encodings.append(face_recognition.face_encodings(x)[0])
    except IndexError:
        print("No face detected.")
        continue


results = []

for i in range(len(unknown_encodings)):
    results.append(face_recognition.compare_faces(labelled_face_encodings, unknown_encodings[i]))

for x in results:
    print("Subject matches: " + str(x))



No face detected.
No face detected.
No face detected.
Subject matches: [False, True, False, False, False, False, False, False, False, False, False, False, False, False, False]
Subject matches: [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]
Subject matches: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]
Subject matches: [False, False, False, True, False, True, False, False, False, False, False, False, False, True, False]
Subject matches: [False, False, False, True, False, True, False, False, False, False, True, False, False, True, False]
Subject matches: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, True]
Subject matches: [False, False, True, False, False, False, False, False, False, False, False, False, False, False, False]
Subject matches: [False, True, False, False, False, False, False, False, False, False, False, Fal

In [7]:
# KNN Classifier

import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'PNG'}

def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    X = []
    y = []

    # Loop through each person in the training set
    for class_dir in os.listdir(train_dir):
        
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        if class_dir == 'all':
            continue

        if class_dir == '.DS_Store':
            continue

        # Loop through each training image for the current person
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                # If there are no people (or too many people) in a training image, skip the image.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Add face encoding for current image to the training set
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    # Determine how many neighbors to use for weighting in the KNN classifier
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf


def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.6):

    if X_img_path == "/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces/all/.DS_Store":
        return []

    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    # Load a trained KNN model (if one was passed in)
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    # Load image file and find face locations
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # If no faces are found in the image, return an empty result.
    if len(X_face_locations) == 0:
        return []

    # Find encodings for faces in the test iamge
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Use the KNN model to find the best matches for the test face
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Predict classes and remove classifications that aren't within the threshold
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]

def show_prediction_labels_on_image(img_path, predictions):

    if img_path == "/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces/all/.DS_Store":
        return []

    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        # Draw a box around the face using the Pillow module
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        # There's a bug in Pillow where it blows up with non-UTF-8 text
        # when using the default bitmap font
        name = name.encode("UTF-8")

        # Draw a label with a name below the face
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    # Remove the drawing library from memory as per the Pillow docs
    del draw

    # Display the resulting image
    pil_image.show()


classifier = train("/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces", model_save_path=None, n_neighbors=None)

for image_file in os.listdir("/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces/all"):
    full_file_path = os.path.join("/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces/all", image_file)

    predictions = predict(full_file_path, knn_clf= classifier)

    for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))
    
    show_prediction_labels_on_image(os.path.join("/Users/ahood/Code/data_science_projects/projects/facial_recognition/faces/all", image_file), predictions)


- Found 2 at (98, 81)
- Found 3 at (96, 96)
- Found 15 at (96, 96)
- Found 14 at (67, 82)
- Found 14 at (67, 82)
- Found 15 at (110, 96)
- Found 3 at (96, 96)
- Found 2 at (124, 82)
- Found 3 at (110, 96)
- Found 14 at (67, 96)
- Found 15 at (96, 96)
- Found 15 at (96, 82)
- Found 14 at (67, 82)
- Found 3 at (96, 96)
- Found 2 at (124, 96)
- Found 1 at (124, 82)
- Found 9 at (110, 82)
- Found 14 at (67, 82)
- Found 10 at (110, 67)
- Found 10 at (124, 82)
- Found 14 at (67, 82)
- Found 9 at (96, 82)
- Found 2 at (139, 82)
- Found 2 at (124, 82)
- Found 3 at (96, 96)
- Found 12 at (115, 81)
- Found 15 at (110, 96)
- Found 14 at (67, 96)
- Found 14 at (67, 82)
- Found 15 at (110, 82)
- Found 12 at (115, 64)
- Found 3 at (110, 96)
- Found 2 at (124, 96)
- Found 8 at (96, 96)
- Found 9 at (96, 82)
- Found 6 at (38, 82)
- Found 5 at (124, 82)
- Found 15 at (110, 96)
- Found 10 at (110, 82)
- Found 13 at (110, 96)
- Found 13 at (127, 116)
- Found 10 at (124, 82)
- Found 15 at (96, 82)
- Found

In [10]:
# OpenCV Face Recognition

# Create CSV file

BASE_PATH="/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces"
SEPARATOR=","

label = 0
for dirname, dirnames, filenames in os.walk(BASE_PATH):
    for subdirname in dirnames:
        subject_path = os.path.join(dirname, subdirname)
        for filename in os.listdir(subject_path):
            abs_path = "%s/%s" % (subject_path, filename)
            print ("%s%s%d" % (abs_path, SEPARATOR, label))
        label = label + 1

/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/7.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/6.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/4.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/5.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/1.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/2.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/3.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/8.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/9.pgm,0
/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV/openCV_faces/s34/10.pgm,0

In [29]:
# Eigenfaces Method

output_folder = '.'
csv_path = "/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV_faces.csv"
images = []
labels = []
paths = []
predictions = []

df = pd.read_csv(csv_path)

paths = df["path"].tolist()
labels = df["label"].tolist()

for x in paths:
    image_local = cv2.imread(x,0)
    images.append(image_local)

x_train,x_test,y_train,y_test = train_test_split(images,labels, test_size=0.2)


y_train = numpy.asarray(y_train)
y_test = numpy.asarray(y_test)

model = cv2.face.EigenFaceRecognizer_create()
model.train(x_train,y_train)

for x in x_test:
    prediciton = model.predict(x)
    predictions.append(prediciton[0])

print(accuracy_score(y_test,predictions))







0.95


In [31]:
#Fisher Faces
output_folder = '.'
csv_path = "/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV_faces.csv"
images = []
labels = []
paths = []
predictions = []

df = pd.read_csv(csv_path)

paths = df["path"].tolist()
labels = df["label"].tolist()

for x in paths:
    image_local = cv2.imread(x,0)
    images.append(image_local)

x_train,x_test,y_train,y_test = train_test_split(images,labels, test_size=0.2)


y_train = numpy.asarray(y_train)
y_test = numpy.asarray(y_test)

model = cv2.face.FisherFaceRecognizer_create()
model.train(x_train,y_train)

for x in x_test:
    prediciton = model.predict(x)
    predictions.append(prediciton[0])

print(accuracy_score(y_test,predictions))


0.9375


In [32]:
# Local Binary Patterns Histograms Method
output_folder = '.'
csv_path = "/Users/ahood/Code/data_science_projects/projects/facial_recognition/openCV_faces.csv"
images = []
labels = []
paths = []
predictions = []

df = pd.read_csv(csv_path)

paths = df["path"].tolist()
labels = df["label"].tolist()

for x in paths:
    image_local = cv2.imread(x,0)
    images.append(image_local)

x_train,x_test,y_train,y_test = train_test_split(images,labels, test_size=0.2)


y_train = numpy.asarray(y_train)
y_test = numpy.asarray(y_test)

model = cv2.face.LBPHFaceRecognizer_create()
model.train(x_train,y_train)

for x in x_test:
    prediciton = model.predict(x)
    predictions.append(prediciton[0])

print(accuracy_score(y_test,predictions))

0.975
